In [1]:
import tensorflow

import pandas as pd
import numpy as np #데이터 배열화
import os #경로 설정용 
import keras #딥러닝용 패키지
import random #데이터 분산할 때 쓸 랜덤
import cv2 #이미지 읽기용
import math #연산용
import seaborn as sns #matplotlib에 다양한 시각화 기능이 추가된 패키지

from sklearn.metrics import confusion_matrix #분류의 정확성 평가
from sklearn.preprocessing import LabelBinarizer #데이터 전처리용
from sklearn.model_selection import train_test_split #데이터 분할용

import matplotlib.pyplot as plt #데이터 시각화용

from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Convolution2D,BatchNormalization #사용할 BN, ConV2 등의 계층
from tensorflow.keras.layers import Flatten,MaxPooling2D,Dropout #사용할 레이어

from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.applications.densenet import preprocess_input #tensor나 numpy배열 전처리용

from tensorflow.keras.preprocessing import image #이미지 데이터를 실시간으로 처리하기 위한 도구
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array #이미지 편집을 위한 제너레이터(Affine Transform)

from tensorflow.keras.models import Model #교육 및 추론 기능이 있는 개체로 레이어를 그룹화

from tensorflow.keras.optimizers import Adam #Adam 옵티마이저 사용. loss는 categorical_crossentropy 사용

#체크포인트를 두고 저장 + metric이 중지되면 학습률을 감소
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau 

import warnings #경고 무시
warnings.filterwarnings("ignore")

In [2]:
data=[]
labels=[]
random.seed(42)
imagePaths = sorted(list(os.listdir("emotion_FixedVer/")))
random.shuffle(imagePaths)
print(imagePaths)

for img in imagePaths:
    path=sorted(list(os.listdir("emotion_FixedVer/"+img)))
    for i in path:
        image = cv2.imread("emotion_FixedVer/"+img+'/'+i)
        image = cv2.resize(image, (48,48))
        image = img_to_array(image)
        data.append(image)
        l = label = img
        labels.append(l)

['neutral', 'fearful', 'happy', 'sad', 'angry', 'surprised']


In [3]:
data = np.array(data, dtype="float32") / 255.0
labels = np.array(labels)
mlb = LabelBinarizer()
labels = mlb.fit_transform(labels)
print(labels[0])

[0 0 0 1 0 0]


In [4]:
(xtrain,xtest,ytrain,ytest)=train_test_split(data,labels,test_size=0.2,random_state=42)
print(xtrain.shape, xtest.shape)

(22294, 48, 48, 3) (5574, 48, 48, 3)


In [5]:
datagen = ImageDataGenerator(zoom_range = 0.2, horizontal_flip=True, shear_range=0.2) #이미지 가공(학습률 향상을 위한 augmentation)
datagen.fit(xtrain)

In [6]:
xtrain[0].shape

(48, 48, 3)

In [7]:
model_d=DenseNet201(weights='imagenet',include_top=False, input_shape=(48, 48, 3)) #채널이 무조건 3개여야 하며 크기는 최소 32 이상

x=model_d.output

x= GlobalAveragePooling2D()(x) #전역 평균 풀링 레이어 추가
x= BatchNormalization()(x) #배치 정규화 레이어
x= Dropout(0.3)(x)
#Fully Connected 레이어 추가
x= Dense(1024,activation='relu')(x) 
x= Dense(512,activation='relu')(x) 
x= BatchNormalization()(x)
x= Dropout(0.3)(x) #과적합 감소용 드롭아웃 레이어

output=Dense(6,activation='softmax')(x) # because we have to predict the AUC
model=Model(model_d.input,output)

In [8]:
model.compile(
    optimizer = Adam(lr=0.1), 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
  )

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 48, 48, 3)]  0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 54, 54, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 24, 24, 64)   9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 24, 24, 64)   256         conv1/conv[0][0]                 
______________________________________________________________________________________________

In [9]:
history = model.fit(datagen.flow(xtrain, ytrain, batch_size=32),
               epochs=50,
               validation_data=(xtest, ytest))

Epoch 1/50
697/697 [==============================] - 131s 156ms/step - loss: 2.5723 - accuracy: 0.1966 - val_loss: 717470.4375 - val_accuracy: 0.1722
Epoch 2/50
697/697 [==============================] - 103s 148ms/step - loss: 2.9357 - accuracy: 0.1881 - val_loss: 2.4911 - val_accuracy: 0.2610
Epoch 3/50
697/697 [==============================] - 107s 153ms/step - loss: 2.5297 - accuracy: 0.1921 - val_loss: 8292.4043 - val_accuracy: 0.1629
Epoch 4/50
697/697 [==============================] - 105s 150ms/step - loss: 2.5481 - accuracy: 0.1957 - val_loss: 74800.4844 - val_accuracy: 0.1720
Epoch 5/50
697/697 [==============================] - 103s 148ms/step - loss: 2.1717 - accuracy: 0.2009 - val_loss: 10550.7881 - val_accuracy: 0.2558
Epoch 6/50
697/697 [==============================] - 105s 151ms/step - loss: 2.5477 - accuracy: 0.1944 - val_loss: 20.8692 - val_accuracy: 0.1652
Epoch 7/50
697/697 [==============================] - 103s 148ms/step - loss: 2.0296 - accuracy: 0.2026 - v

In [10]:
model.evaluate(xtest, ytest, batch_size = 32)

175/175 [==============================] - 8s 46ms/step - loss: 315972768.0000 - accuracy: 0.4424


[315972768.0, 0.4424111843109131]

In [11]:
model.save('Densenet201_keras_emotion6ver_imsize48__lr0.1_epoch50.h5')

In [12]:
#완성도 확인
ypred = model.predict(xtest)

total = 0
accurate = 0
accurateindex = []
wrongindex = []

for i in range(len(ypred)):
    if np.argmax(ypred[i]) == np.argmax(ytest[i]): #가장 큰 값을 찾아 인덱스 반환
        accurate += 1
        accurateindex.append(i)
    else:
        wrongindex.append(i)
        
    total += 1
    
print('Total-test-data;', total, '\taccurately-predicted-data:', accurate, '\t wrongly-predicted-data: ', total - accurate)
print('Accuracy:', round(accurate/total*100, 3), '%')

Total-test-data; 5574 	accurately-predicted-data: 2466 	 wrongly-predicted-data:  3108
Accuracy: 44.241 %
